## Imports

In [ ]:
import numpy as np
import meshplot as mp

import wildmeshing as wm
import polyfempy as pf

## Load Data

In [ ]:
V = np.load("V.npy")
L = np.load("L.npy")

p=mp.plot(V, np.zeros((0,3)), return_plot=True)
p.add_edges(V, L, shading={});
p.add_points(V, shading={"point_color": "red", "point_size": 10})

## Meshing

In [ ]:
vs, tris, nodes, F_nodes = wm.triangulate_svg("plane_hole.svg", cut_outside=True)

In [ ]:
p = mp.plot(vs, tris, shading={"wireframe": True}, return_plot=True)
p.add_points(V, shading={"point_color": "red", "point_size": 2});
p.add_points(nodes, shading={"point_color": "green", "point_size": 2})

## Sidesets for Boundary Conditions

In [ ]:
solver = pf.Solver()

In [ ]:
solver.set_high_order_mesh(vs, tris, nodes, F_nodes)

sp, sl, s = solver.get_boundary_sidesets()

In [ ]:
p=mp.plot(sp, np.zeros((0,3)), return_plot=True)

p.add_edges(sp, sl);

e1 = sl[s[:,0] == 1, :]
e2 = sl[s[:,0] == 2, :]
e3 = sl[s[:,0] == 3, :]
e4 = sl[s[:,0] == 4, :]

p.add_points((sp[e1[:, 0], :]+sp[e1[:, 1], :])/2, shading={"point_color": "red", "point_size": 0.1});
p.add_points((sp[e2[:, 0], :]+sp[e2[:, 1], :])/2, shading={"point_color": "green", "point_size": 0.1});
p.add_points((sp[e3[:, 0], :]+sp[e3[:, 1], :])/2, shading={"point_color": "blue", "point_size": 0.1});
p.add_points((sp[e4[:, 0], :]+sp[e4[:, 1], :])/2, shading={"point_color": "black", "point_size": 0.1})

## Problem and Settings Setup

In [ ]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.NonLinearElasticity)

settings.discr_order = 2

settings.set_material_params("E", 210000)
settings.set_material_params("nu", 0.3)

problem.set_x_symmetric(1)
problem.set_y_symmetric(2)
problem.set_force(3, [100, 0])

settings.set_problem(problem)

## Solving!

In [ ]:
solver.settings(settings)
solver.solve()

## Results

In [ ]:
sol_pts, sol_tri, disp = solver.get_sampled_solution()
misises, _ = solver.get_sampled_mises_avg()

In [ ]:
mp.plot(sol_pts+disp, sol_tri, misises)